<a href="https://colab.research.google.com/github/abhinavgairola/Anomaly_Detection_OTC_Market/blob/main/Scripts/Download_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Read the tweets from downloaded json files and store it into a leaner format in a parquet file.

In [1]:
import pandas_datareader as pdr
import pandas as pd
import datetime
import matplotlib.pyplot as mp
import re
import json
from google.colab import drive
drive.mount('/content/drive')
import csv
import pyarrow as pa

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
%cd drive

/content/drive


In [4]:
%cd MyDrive/OTCMKT/

/content/drive/MyDrive/OTCMKT


In [5]:
!ls

a2.html					       OTC.png
Adjusted_close_no_na.parquet.gzip	       Stock_Screener.csv
All_tweets_correct_format.parquet.gzip	       Stocks_data_2.parquet.gzip
All_tweets.json				       Stocks_data_3.parquet.gzip
All_tweet_stocks.csv			       Stocks_data.parquet.gzip
All_tweet_stocks.parquet.gzip		       table.html
All_tweets_with_emojis_with_punctuations.json  Tickers.txt
Din_Condensed.ttf			       Tweets_json
Final_data.csv				       Tweets.zip
OTC2.jpeg				       Word_Cloud_2.png
OTC3.png				       Word_Cloud.png
OTC.jpeg


In [27]:
f = open('Stock_Screener.csv','r')
Tickers = []
for i,line in enumerate(f):
  line_split = line.split(',')
  #print(line_split)
  if i !=0:
    Tickers.append(line_split[0])
#print(Tickers)

In [26]:
df = pd.read_csv('Stock_Screener.csv')
#df = df[df['Change %']>1]
df = df[df['Vol']>0]
df_2=df.filter(items =['Symbol','Tier','Country','State','Change %','Vol','Price'])
df_USA=df_2#[df_2['Country']=='USA']
df_USA=df_USA.sort_values('Vol',ascending=False)
#df_USA = df_USA[df_USA['Change %']]
#df_USA['Tier'].unique()
Tickers = df_USA['Symbol']
Tickers=list(Tickers.values)
list(df_USA.iloc[0:10:]['Country'].unique())

['USA']

In [11]:
new = pdr.data.DataReader(Tickers[0:1000],'yahoo',start='01/01/2016')#,end='04/10/2021')
table = pa.Table.from_pandas(new)
pa.parquet.write_table(table, '/content/drive/MyDrive/OTCMKT/Stocks_data_3.parquet.gzip')


In [12]:
df_stocks = pd.read_parquet('/content/drive/MyDrive/OTCMKT/Stocks_data_3.parquet.gzip')

In [13]:
indexes = (df_stocks['Adj Close'].isnull().mean()<=0.5)
indexes = (df_stocks['Adj Close'].isnull().mean()<=0.5)
df_stocks_adjusted_close = (df_stocks['Adj Close'][df_stocks['Adj Close'].columns[indexes]]).fillna(0)
df_stocks_adjusted_close
table_adjusted = pa.Table.from_pandas(df_stocks_adjusted_close)
pa.parquet.write_table(table_adjusted, '/content/drive/MyDrive/OTCMKT/Adjusted_close_no_na.parquet.gzip')

In [14]:
#columns  = list(df_stocks['Adj Close'].columns)

#df_stocks['Adj Close']#[columns[0:10]].plot(figsize=(20,10))

In [15]:
#import emoji
!pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.7/dist-packages (1.2.0)


In [16]:
import emoji

In [17]:
import glob 
import itertools
filenames = glob.glob('/content/drive/MyDrive/OTCMKT/Tweets_json/*.json')
print(itertools.chain(filenames))
import re
import string
emoj_pattern = emoji.get_emoji_regexp()
def remove_emojis(data,emoj):
    #emoj = emoji.get_emoji_regexp()
    return re.sub(emoj, '', data)

All_tweets={}
for it in itertools.chain(filenames):
  #print(it)
  date = []
  tweet = []
  unique = (it.split('/')[-1].split('.')[0].split('$')[-1])
  for i,f in enumerate(open(it,'r')):
      data = json.loads(f)
      date.append(data['date'])
      #tweet.append(re.sub('[\#+|\-+|\$+|\_+|\.+]','',remove_emojis(data['tweet'],emoj_pattern)))
      tweet.append(re.sub('[\#+|\-+|\$+|\_+|\.+]','',data['tweet']))
  All_tweets['Date'+unique]=date
  All_tweets['Tweets'+unique] = tweet
#All_tweets

In [18]:
key_list = list(All_tweets.keys())
pattern = re.compile('https?://[A-Za-z0-9./]+')
pattern_2 = re.compile('@[A-Za-z0-9]+')
pattern_3 = re.compile(r'\———+')
for ke in key_list:
  tweets = []
  if ke.find('Tweets')!=-1:
    vals = All_tweets[ke]
    for tweet in vals:
      tweets.append((re.sub(pattern_3,'',re.sub(pattern_2,'',re.sub(pattern,'',tweet)))).replace(string.punctuation,''))
    All_tweets[ke]=tweets
#All_tweets

In [19]:
#All_tweets['TweetsRGBP']

In [20]:
with open('/content/drive/MyDrive/OTCMKT/All_tweets_with_emojis_with_punctuations.json', 'w') as outfile:
    json.dump(All_tweets, outfile)

# This I am doing it to bring it to proper shape.

In [21]:
with open('/content/drive/MyDrive/OTCMKT/All_tweets_with_emojis_with_punctuations.json','r') as f:
  data = (json.load(f))


df = pd.DataFrame.from_dict(data, orient='index')
print(df)
Dates = []
Tweets = []
Ticker  = []
for key, vals in data.items():
  if key.find('Date')>-1:
    Dates = Dates+vals
  elif key.find('Tweets')>-1:
    Tweets = Tweets+vals
    Ticker = Ticker + [key.split('Tweets')[1]]*len(vals)
df_tweets = pd.DataFrame({'Dates':Dates,'Tweets':Tweets,'Ticker':Ticker})
## Convert dates column to datetime index
df_tweets['Dates'] = pd.to_datetime(df_tweets['Dates'])
df_tweets.set_index('Dates')[['Ticker','Tweets']].to_parquet('/content/drive/MyDrive/OTCMKT/All_tweets_correct_format.parquet.gzip',
              compression='gzip')

                                                       0       ... 136113
DateXTRM                                           2021-04-10  ...   None
TweetsXTRM                        XTRM Which house  you like?  ...   None
DateEESO                                           2021-04-08  ...   None
TweetsEESO   When will we hear something Is EESO going to ...  ...   None
DateSKGO                                           2021-03-31  ...   None
...                                                       ...  ...    ...
TweetsSPRL  Hey guys! This is the Twitter account of the s...  ...   None
DateRXMD                                           2021-04-10  ...   None
TweetsRXMD  RXMD Most undervalued OTC on the market 012/sh...  ...   None
DateDKTS                                           2021-03-26  ...   None
TweetsDKTS  MustWatch   FDBL  NEWS     OTC OTCMarkets TRAD...  ...   None

[2712 rows x 136114 columns]


In [23]:
df_tweets = pd.read_parquet('/content/drive/MyDrive/OTCMKT/All_tweets_correct_format.parquet.gzip')

In [24]:
df_tweets.shape

(10858927, 2)